Importing the required packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import torch
import torch.nn.functional as F
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from IPython.display import Image, display
import matplotlib.image as img
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Importing the datasets

In [ ]:
path_train = '/content/gdrive/MyDrive/ai-unibuc-23-31-2021/train'
path_val = '/content/gdrive/MyDrive/ai-unibuc-23-31-2021/validation'
path_test = '/content/gdrive/MyDrive/ai-unibuc-23-31-2021/test'
train_data = pd.read_csv('/content/gdrive/MyDrive/ai-unibuc-23-31-2021/train.txt', sep =",", names=['path', 'label'])
val_data = pd.read_csv('/content/gdrive/MyDrive/ai-unibuc-23-31-2021/validation.txt', sep =",", names=['path', 'label'])
test_data = pd.read_csv('/content/gdrive/MyDrive/ai-unibuc-23-31-2021/test.txt', sep =",", names=['path', 'label'])

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

Transforming and normalizing the images

In [ ]:
transformer = transforms.Compose([
                        transforms.ToPILImage(),
                        transforms.ToTensor(),
                        transforms.Normalize((0.5,), (0.5,))])

In [ ]:
train_set = []
val_set = []
test_set = []
for i in range(len(train_data)):
  img_path, label = train_data['path'][i], train_data['label'][i]
  img_path = os.path.join(path_train, train_data['path'][i])
  image = img.imread(img_path)
  image = transformer(image)
  train_set.append([image, label])

for i in range(len(val_data)):
  img_path, label = val_data['path'][i], val_data['label'][i]
  img_path = os.path.join(path_val, val_data['path'][i])
  image = img.imread(img_path)
  image = transformer(image)
  val_set.append([image, label])

# for i in range(len(test_data)):
#   img_path = test_data['path'][i]
#   img_path = os.path.join(path_test, test_data['path'][i])
#   image = img.imread(img_path)
#   image = transformer(image)
#   test_set.append(image)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:132: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  img = torch.from_numpy(np.array(pic, np.float32, copy=False))


Creating the dataloaders

In [ ]:
batch_size = 100
batch_size1 = len(test_data)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True,drop_last=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True,drop_last=True)
test_loader = DataLoader(test_set, batch_size=batch_size1)
first_train_batch_imgs, first_train_batch_labels = next(iter(train_loader))

Defining the CNN model

In [ ]:
class CNN(nn.Module):
    def __init__(self):
      super().__init__()
      self.conv1 = nn.Conv2d(1, 15, 3, 1, padding=1)
      self.conv2 = nn.Conv2d(15, 30, 3, 1, padding=1)
      self.conv3 = nn.Conv2d(30, 50, 3, 1, padding=1)
      self.fc1 = nn.Linear(6*6*50, 100)
      self.dropout1 = nn.Dropout(0.5)
      self.fc2 = nn.Linear(100, 3)
    def forward(self, x):
      x = F.relu(self.conv1(x))
      x = F.max_pool2d(x, 2, 2)
      x = F.relu(self.conv2(x))
      x = F.max_pool2d(x, 2, 2)
      x = F.relu(self.conv3(x))
      x = F.max_pool2d(x, 2, 2)
      x = x.view(-1, 6*6*50)
      x = F.relu(self.fc1(x))
      x = self.dropout1(x)
      x = self.fc2(x)
      return x

Defining two functions: one for training, one for testing

In [ ]:
def train(model, device, train_loader):
    model.train()
    train_losses = []
    for index, (image, label) in enumerate(train_loader):
        image, label = image.to(device), label.to(device)
        output = model(image)
        loss = criterion(output, label)
  
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_losses.append(loss.detach().cpu().numpy())
        if index % 150 == 0:
            print('epoch:' + str(epoch))
            print('train loss:' + str(loss.item()))
    return sum(train_losses)/len(train_losses)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    corrects = 0
    prediction = []
    labels = []
    with torch.no_grad():
        n = 0
        for image, label in test_loader:
            image, label = image.to(device), label.to(device)
            output = model(image)
            loss = criterion(output, label) 
            test_loss += loss

            _, pred = torch.max(output, 1)
            corrects += torch.sum(pred == label.data)
            pred = pred.detach().cpu().numpy()
            prediction.append(pred)
            label = label.detach().cpu().numpy()
            labels.append(label)
            n += 1

    test_loss /= n
    test_accuracy = corrects / n

    print('test loss:' + str(test_loss.item()))
    print('accuracy:' + str(test_accuracy.item()) +'\n')
    return labels, prediction, test_loss, test_accuracy

Training the model

In [ ]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001, weight_decay = 0.01, amsgrad = True)

losses_train = []
losses_val = []
accuracy_val = []
for epoch in range(25):
    train_loss = train(model, device, train_loader)
    labels, prediction, val_loss, val_accuracy = test(model, device, val_loader)
    losses_train.append(train_loss)
    losses_val.append(val_loss)
    accuracy_val.append(val_accuracy)

plt.figure(1)
plt.plot(losses_train,label='train_loss',color='red')
plt.plot(losses_val,label='val_loss',color='blue')
plt.figure(2)
plt.plot(accuracy_val,label='val_accuracy')

In [ ]:
labels = np.array(labels)
labels = labels.flatten()
prediction = np.array(prediction)
prediction = prediction.flatten()

In [ ]:
print(confusion_matrix(labels,prediction))

[[1242  171   87]
 [ 231  920  349]
 [ 144  326 1030]]


Creating a predictions document

In [ ]:
predictions = []
with torch.no_grad():
  for data in test_loader:
    data = data.to(device)
    print(data)
    output = model(data) 
    pred = output.argmax(1)
    predictions.append(pred.cpu().detach().numpy())

In [ ]:
import numpy as np
predictions = np.array(predictions)
pred2 = []
for i in predictions:
    pred2.append(i)
predictions = predictions.flatten()

fout = open('/content/gdrive/MyDrive/ai-unibuc-23-31-2021/submission.txt', 'w')
fout.write('id,label\n')
for index, row in test_data.iterrows():
    fout.write(row['path'] + ',' + str(pred2[index]) + '\n')
fout.close()